# End-to-End Document Processing with IDP Common Package

This notebook demonstrates how to process a document using the modular Document-based approach with:

1. OCR Service - Convert a PDF document to text using AWS Textract
2. Classification Service - Classify document pages into sections using Bedrock using the multi-model page based method.
3. Extraction Service - Extract structured information from sections using Bedrock
4. Evaluation Service - Evaluate accuracy of extracted information

Each step uses the unified Document object model for data flow and consistency.

> **Note**: This notebook uses AWS services including S3, Textract, and Bedrock. You need valid AWS credentials with appropriate permissions to run this notebook.

## 1. Install Dependencies

The IDP common package supports granular installation through extras. You can install:
- `[core]` - Just core functionality 
- `[ocr]` - OCR service with Textract dependencies
- `[classification]` - Classification service dependencies
- `[extraction]` - Extraction service dependencies
- `[evaluation]` - Evaluation service dependencies
- `[all]` - All of the above

In [1]:
# Let's make sure that modules are autoreloaded
%load_ext autoreload
%autoreload 2

ROOTDIR="../.."
# First uninstall existing package (to ensure we get the latest version)
%pip uninstall -y idp_common

# Install the IDP common package with all components in development mode
%pip install -q -e "{ROOTDIR}/lib/idp_common_pkg[dev, all]"

# Note: We can also install specific components like:
# %pip install -q -e "{ROOTDIR}/lib/idp_common_pkg[ocr,classification,extraction,evaluation]"

# Check installed version
%pip show idp_common | grep -E "Version|Location"

# Optionally use a .env file for environment variables
try:
    from dotenv import load_dotenv
    load_dotenv()  
except ImportError:
    pass  

Found existing installation: idp_common 0.3.4
Uninstalling idp_common-0.3.4:
  Successfully uninstalled idp_common-0.3.4
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Version: 0.3.4
Location: /home/ec2-user/.local/lib/python3.11/site-packages
Note: you may need to restart the kernel to use updated packages.


## 2. Import Libraries and Set Up Environment

In [ ]:
import os
import json
import time
import boto3
import logging
import datetime

# Import base libraries
from idp_common.models import Document, Status, Section, Page
from idp_common import ocr, classification, extraction, evaluation

# Configure logging 
logging.basicConfig(level=logging.WARNING)  # Set root logger to WARNING (less verbose)
logging.getLogger('idp_common.ocr.service').setLevel(logging.INFO)  # Focus on service logs
logging.getLogger('idp_common.classification.service').setLevel(logging.DEBUG)  # Enable classification logs
logging.getLogger('idp_common.bedrock.client').setLevel(logging.DEBUG)  # show prompts

logging.getLogger('idp_common.evaluation.service').setLevel(logging.DEBUG)  # Enable evaluation logs

logging.getLogger('textractor').setLevel(logging.WARNING)  # Suppress textractor logs
logging.getLogger('idp_common.evaluation.service').setLevel(logging.DEBUG)  # Enable evaluation logs

# Set environment variables
os.environ['METRIC_NAMESPACE'] = 'IDP-Notebook-Example'
os.environ['AWS_REGION'] = boto3.session.Session().region_name or 'us-east-1'

# Get AWS account ID for unique bucket names
sts_client = boto3.client('sts')
account_id = sts_client.get_caller_identity()["Account"]
region = os.environ['AWS_REGION']

# Define sample PDF path 
SAMPLE_PDF_PATH = f"{ROOTDIR}/samples/rvl_cdip_package.pdf"

# Create unique bucket names based on account ID and region
input_bucket_name =  os.getenv("IDP_INPUT_BUCKET_NAME", f"idp-notebook-input-{account_id}-{region}")
output_bucket_name = os.getenv("IDP_OUTPUT_BUCKET_NAME", f"idp-notebook-output-{account_id}-{region}")

# Helper function to parse S3 URIs
def parse_s3_uri(uri):
    parts = uri.replace("s3://", "").split("/")
    bucket = parts[0]
    key = "/".join(parts[1:])
    return bucket, key

# Helper function to load JSON from S3
def load_json_from_s3(uri):
    bucket, key = parse_s3_uri(uri)
    response = s3_client.get_object(Bucket=bucket, Key=key)
    content = response['Body'].read().decode('utf-8')
    return json.loads(content)

# Set ROOT_DIR - used to locate example images from local directory
# OR set CONFIGURATION_BUCKET to S3 Configration bucket name (contains config_library)
os.environ['ROOT_DIR'] = ROOTDIR

print("Environment setup:")
print(f"METRIC_NAMESPACE: {os.environ.get('METRIC_NAMESPACE')}")
print(f"AWS_REGION: {os.environ.get('AWS_REGION')}")
print(f"Input bucket: {input_bucket_name}")
print(f"Output bucket: {output_bucket_name}")
print(f"SAMPLE_PDF_PATH: {SAMPLE_PDF_PATH}")

Environment setup:
METRIC_NAMESPACE: IDP-Notebook-Example
AWS_REGION: us-west-2
Input bucket: idp-notebook-input-912625584728-us-west-2
Output bucket: idp-notebook-output-912625584728-us-west-2
SAMPLE_PDF_PATH: ../samples/rvl_cdip_package.pdf


## 3. Set Up S3 Buckets and Upload Sample File

In [3]:
# Create S3 client
s3_client = boto3.client('s3')

# Function to create a bucket if it doesn't exist
def ensure_bucket_exists(bucket_name):
    try:
        s3_client.head_bucket(Bucket=bucket_name)
        print(f"Bucket {bucket_name} already exists")
    except Exception:
        try:
            if region == 'us-east-1':
                s3_client.create_bucket(Bucket=bucket_name)
            else:
                s3_client.create_bucket(
                    Bucket=bucket_name,
                    CreateBucketConfiguration={'LocationConstraint': region}
                )
            print(f"Created bucket: {bucket_name}")
            
            # Wait for bucket to be accessible
            waiter = s3_client.get_waiter('bucket_exists')
            waiter.wait(Bucket=bucket_name)
        except Exception as e:
            print(f"Error creating bucket {bucket_name}: {str(e)}")
            raise

# Ensure both buckets exist
ensure_bucket_exists(input_bucket_name)
ensure_bucket_exists(output_bucket_name)

# Upload the sample file to S3
sample_file_key = "sample-" + datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S") + ".pdf"
with open(SAMPLE_PDF_PATH, 'rb') as file_data:
    s3_client.upload_fileobj(file_data, input_bucket_name, sample_file_key)

print(f"Uploaded sample file to: s3://{input_bucket_name}/{sample_file_key}")

Bucket idp-notebook-input-912625584728-us-west-2 already exists
Bucket idp-notebook-output-912625584728-us-west-2 already exists


FileNotFoundError: [Errno 2] No such file or directory: '../samples/rvl_cdip_package.pdf'

## 4. Set Up Configuration

In [ ]:
# Few shot configuration from config_library
import yaml
with open("{ROOTDIR}/config_library/pattern-2/few_shot_example_with_multimodal_page_classification/config.yaml", 'r') as file:
    CONFIG = yaml.safe_load(file)

print("Test configuration created")

Test configuration created


## 5. Process Document with OCR

In [ ]:
# Initialize a new Document
document = Document(
    id="rvl-cdip-package",
    input_bucket=input_bucket_name,
    input_key=sample_file_key,
    output_bucket=output_bucket_name,
    status=Status.QUEUED
)

print(f"Created document with ID: {document.id}")
print(f"Status: {document.status.value}")

# Create OCR service with Textract
# Valid features are 'LAYOUT', 'FORMS', 'SIGNATURES', 'TABLES' (uses analyze_document API)
# or leave it empty (to use basic detect_document_text API)
ocr_service = ocr.OcrService(
    region=region,
    enhanced_features=['LAYOUT']
)

# Process document with OCR
print("\nProcessing document with OCR...")
start_time = time.time()
document = ocr_service.process_document(document)
ocr_time = time.time() - start_time

print(f"OCR processing completed in {ocr_time:.2f} seconds")
print(f"Document status: {document.status.value}")
print(f"Number of pages processed: {document.num_pages}")

# Show pages information
print("\nProcessed pages:")
for page_id, page in document.pages.items():
    print(f"Page {page_id}:")
    print(f"  Image URI: {page.image_uri}")
    print(f"  Raw Text URI: {page.raw_text_uri}")
    print(f"  Parsed Text URI: {page.parsed_text_uri}")
print("\nMetering:")
print(json.dumps(document.metering))

INFO:idp_common.ocr.service:OCR Service initialized with features: ['LAYOUT']


Created document with ID: rvl-cdip-package
Status: QUEUED

Processing document with OCR...


INFO:idp_common.ocr.service:Successfully extracted markdown text for page 7
INFO:idp_common.ocr.service:Successfully extracted markdown text for page 9
INFO:idp_common.ocr.service:Successfully extracted markdown text for page 3
INFO:idp_common.ocr.service:Successfully extracted markdown text for page 5
INFO:idp_common.ocr.service:Successfully extracted markdown text for page 10
INFO:idp_common.ocr.service:Successfully extracted markdown text for page 8
INFO:idp_common.ocr.service:Successfully extracted markdown text for page 2
INFO:idp_common.ocr.service:Successfully extracted markdown text for page 6
INFO:idp_common.ocr.service:Successfully extracted markdown text for page 1
INFO:idp_common.ocr.service:Successfully extracted markdown text for page 4
INFO:idp_common.ocr.service:Sorting 10 pages by page number
INFO:idp_common.ocr.service:OCR processing completed in 6.15 seconds
INFO:idp_common.ocr.service:Processed 10 pages, with 0 errors


OCR processing completed in 6.16 seconds
Document status: QUEUED
Number of pages processed: 10

Processed pages:
Page 1:
  Image URI: s3://idp-notebook-output-912625584728-us-west-2/sample-2025-05-27_14-24-41.pdf/pages/1/image.jpg
  Raw Text URI: s3://idp-notebook-output-912625584728-us-west-2/sample-2025-05-27_14-24-41.pdf/pages/1/rawText.json
  Parsed Text URI: s3://idp-notebook-output-912625584728-us-west-2/sample-2025-05-27_14-24-41.pdf/pages/1/result.json
Page 2:
  Image URI: s3://idp-notebook-output-912625584728-us-west-2/sample-2025-05-27_14-24-41.pdf/pages/2/image.jpg
  Raw Text URI: s3://idp-notebook-output-912625584728-us-west-2/sample-2025-05-27_14-24-41.pdf/pages/2/rawText.json
  Parsed Text URI: s3://idp-notebook-output-912625584728-us-west-2/sample-2025-05-27_14-24-41.pdf/pages/2/result.json
Page 3:
  Image URI: s3://idp-notebook-output-912625584728-us-west-2/sample-2025-05-27_14-24-41.pdf/pages/3/image.jpg
  Raw Text URI: s3://idp-notebook-output-912625584728-us-west-2/s

## 6. Classify the Document

In [ ]:
# Verify that Config specifies => "classificationMethod": "textbasedHolisticClassification"
print("*****************************************************************")
print(f'CONFIG classificationMethod: {CONFIG["classification"].get("classificationMethod")}')
print("*****************************************************************")

# Create classification service with Bedrock backend
# The classification method is set in the config
classification_service = classification.ClassificationService(
    config=CONFIG, 
    backend="bedrock" 
)

# Classify the document
print("\nClassifying document...")
start_time = time.time()
document = classification_service.classify_document(document)
classification_time = time.time() - start_time
print(f"Classification completed in {classification_time:.2f} seconds")
print(f"Document status: {document.status.value}")

INFO:idp_common.classification.service:Initialized classification service with Bedrock backend using model us.amazon.nova-pro-v1:0
INFO:idp_common.classification.service:Using multimodal page-level classification method
INFO:idp_common.classification.service:Classifying document with 10 pages using page-by-page method with bedrock backend


*****************************************************************
CONFIG classificationMethod: multimodalPageLevelClassification
*****************************************************************

Classifying document...


INFO:idp_common.classification.service:Classifying page 10 with Bedrock
INFO:idp_common.classification.service:Classifying page 3 with Bedrock
INFO:idp_common.classification.service:Classifying page 5 with Bedrock
INFO:idp_common.classification.service:Classifying page 7 with Bedrock
INFO:idp_common.classification.service:Classifying page 9 with Bedrock
INFO:idp_common.classification.service:Classifying page 2 with Bedrock
INFO:idp_common.classification.service:Classifying page 8 with Bedrock
INFO:idp_common.classification.service:Classifying page 1 with Bedrock
INFO:idp_common.classification.service:Classifying page 4 with Bedrock
INFO:idp_common.classification.service:Classifying page 6 with Bedrock
DEBUG:idp_common.bedrock.client:No cachepoint tags in text content, passing through unchanged
DEBUG:idp_common.bedrock.client:No cachepoint tags in text content, passing through unchanged
DEBUG:idp_common.bedrock.client:No cachepoint tags in image content, passing through unchanged
DEBUG:

Classification completed in 198.34 seconds
Document status: QUEUED


In [ ]:
# Show classification results
if document.sections:
    print("\nDetected sections:")
    for section in document.sections:
        print(f"Section {section.section_id}: {section.classification}")
        print(f"  Pages: {section.page_ids}")
else:
    print("\nNo sections detected")

# Show page classification
print("\nPage-level classifications:")
for page_id, page in sorted(document.pages.items()):
    print(f"Page {page_id}: {page.classification}")


Detected sections:
Section 1: letter
  Pages: ['1']
Section 2: specification
  Pages: ['2']
Section 3: email
  Pages: ['3']
Section 4: scientific_publication
  Pages: ['4']
Section 5: invoice
  Pages: ['5']
Section 6: advertisement
  Pages: ['6']
Section 7: questionnaire
  Pages: ['7']
Section 8: resume
  Pages: ['8', '9']
Section 9: memo
  Pages: ['10']

Page-level classifications:
Page 1: letter
Page 10: memo
Page 2: specification
Page 3: email
Page 4: scientific_publication
Page 5: invoice
Page 6: advertisement
Page 7: questionnaire
Page 8: resume
Page 9: resume


## 7. Extract Information from Document Sections

In [ ]:
# Create extraction service with Bedrock
extraction_service = extraction.ExtractionService(config=CONFIG)

print("\nExtracting information from document sections...")
extracted_results = {}

n = 3 # Only process first 3 sections to save time
# Process each section directly using the section_id
for section in document.sections[:n]:  
    print(f"\nProcessing section {section.section_id} (class: {section.classification})")
    
    # Process section directly with the original document
    start_time = time.time()
    document = extraction_service.process_document_section(
        document=document,
        section_id=section.section_id
    )
    extraction_time = time.time() - start_time
    print(f"Extraction for section {section.section_id} completed in {extraction_time:.2f} seconds")
    
print(f"\nExtraction for first {n} sections complete.")


Extracting information from document sections...

Processing section 1 (class: letter)


DEBUG:idp_common.bedrock.client:No cachepoint tags in text content, passing through unchanged
DEBUG:idp_common.bedrock.client:No cachepoint tags in text content, passing through unchanged
DEBUG:idp_common.bedrock.client:No cachepoint tags in image content, passing through unchanged
DEBUG:idp_common.bedrock.client:No cachepoint tags in text content, passing through unchanged
DEBUG:idp_common.bedrock.client:No cachepoint tags in image content, passing through unchanged
DEBUG:idp_common.bedrock.client:Found <<CACHEPOINT>> tags in text content: 
</few_shot_examples>
</task>
</background>
<<CACH...
DEBUG:idp_common.bedrock.client:Split text into 2 parts at cachepoint tags
DEBUG:idp_common.bedrock.client:Text part 1: 3 words
DEBUG:idp_common.bedrock.client:Inserting cachePoint #1 after text part 1
DEBUG:idp_common.bedrock.client:Text part 2: 334 words
DEBUG:idp_common.bedrock.client:No cachepoint tags in image content, passing through unchanged
INFO:idp_common.bedrock.client:Processed conten

Extraction for section 1 completed in 27.99 seconds

Processing section 2 (class: specification)


DEBUG:idp_common.bedrock.client:No cachepoint tags in text content, passing through unchanged
DEBUG:idp_common.bedrock.client:Found <<CACHEPOINT>> tags in text content: 
</few_shot_examples>
</task>
</background>
<<CACH...
DEBUG:idp_common.bedrock.client:Split text into 2 parts at cachepoint tags
DEBUG:idp_common.bedrock.client:Text part 1: 3 words
DEBUG:idp_common.bedrock.client:Inserting cachePoint #1 after text part 1
DEBUG:idp_common.bedrock.client:Text part 2: 157 words
DEBUG:idp_common.bedrock.client:No cachepoint tags in image content, passing through unchanged
INFO:idp_common.bedrock.client:Processed content with 1 cachepoint insertions
INFO:idp_common.bedrock.client:Applied cachePoint processing for supported model: us.amazon.nova-pro-v1:0
INFO:idp_common.bedrock.client:Bedrock request attempt 1/8:
DEBUG:idp_common.bedrock.client:  - model: us.amazon.nova-pro-v1:0
DEBUG:idp_common.bedrock.client:  - inferenceConfig: {'temperature': 0.0, 'topP': 0.1, 'maxTokens': 4096}
DEBUG:id

Extraction for section 2 completed in 25.04 seconds

Processing section 3 (class: email)


DEBUG:idp_common.bedrock.client:No cachepoint tags in text content, passing through unchanged
DEBUG:idp_common.bedrock.client:No cachepoint tags in text content, passing through unchanged
DEBUG:idp_common.bedrock.client:No cachepoint tags in image content, passing through unchanged
DEBUG:idp_common.bedrock.client:Found <<CACHEPOINT>> tags in text content: 
</few_shot_examples>
</task>
</background>
<<CACH...
DEBUG:idp_common.bedrock.client:Split text into 2 parts at cachepoint tags
DEBUG:idp_common.bedrock.client:Text part 1: 3 words
DEBUG:idp_common.bedrock.client:Inserting cachePoint #1 after text part 1
DEBUG:idp_common.bedrock.client:Text part 2: 237 words
DEBUG:idp_common.bedrock.client:No cachepoint tags in image content, passing through unchanged
INFO:idp_common.bedrock.client:Processed content with 1 cachepoint insertions
INFO:idp_common.bedrock.client:Applied cachePoint processing for supported model: us.amazon.nova-pro-v1:0
INFO:idp_common.bedrock.client:Bedrock request attem

Extraction for section 3 completed in 8.67 seconds

Extraction for first 3 sections complete.


In [ ]:
print("\nShow extraction results...\n")

document_dict = document.to_dict()
sections_json = json.dumps(document_dict["sections"][:n], indent=2)
print(f"{sections_json}...")


Show extraction results...

[
  {
    "section_id": "1",
    "classification": "letter",
    "confidence": 1.0,
    "page_ids": [
      "1"
    ],
    "extraction_result_uri": "s3://idp-notebook-output-912625584728-us-west-2/sample-2025-05-27_14-24-41.pdf/sections/1/result.json"
  },
  {
    "section_id": "2",
    "classification": "specification",
    "confidence": 1.0,
    "page_ids": [
      "2"
    ],
    "extraction_result_uri": "s3://idp-notebook-output-912625584728-us-west-2/sample-2025-05-27_14-24-41.pdf/sections/2/result.json"
  },
  {
    "section_id": "3",
    "classification": "email",
    "confidence": 1.0,
    "page_ids": [
      "3"
    ],
    "extraction_result_uri": "s3://idp-notebook-output-912625584728-us-west-2/sample-2025-05-27_14-24-41.pdf/sections/3/result.json"
  }
]...


## 8. Final Document Status Summary

In [ ]:
# Update document status to COMPLETED
document.status = Status.COMPLETED

# Display final document state
print("Final Document State:")
print(f"Document ID: {document.id}")
print(f"Status: {document.status.value}")
print(f"Number of pages: {document.num_pages}")
print(f"Number of sections: {len(document.sections)}")

# Show document serialization capabilities
print("\nDocument can be serialized to JSON:")
document_dict = document.to_dict()
document_json = json.dumps(document_dict, indent=2)  
print(f"{document_json}")

Final Document State:
Document ID: rvl-cdip-package
Status: COMPLETED
Number of pages: 10
Number of sections: 9

Document can be serialized to JSON:
{
  "id": "rvl-cdip-package",
  "input_bucket": "idp-notebook-input-912625584728-us-west-2",
  "input_key": "sample-2025-05-27_14-24-41.pdf",
  "output_bucket": "idp-notebook-output-912625584728-us-west-2",
  "status": "COMPLETED",
  "initial_event_time": null,
  "queued_time": null,
  "start_time": null,
  "completion_time": null,
  "workflow_execution_arn": null,
  "num_pages": 10,
  "summary_report_uri": null,
  "evaluation_status": null,
  "evaluation_report_uri": null,
  "errors": [],
  "metering": {
    "textract/analyze_document-Layout": {
      "pages": 10
    },
    "bedrock/us.amazon.nova-pro-v1:0": {
      "inputTokens": 38555,
      "outputTokens": 638,
      "totalTokens": 87862,
      "cacheReadInputTokens": 34919,
      "cacheWriteInputTokens": 13750
    }
  },
  "pages": {
    "1": {
      "page_id": "1",
      "image_uri":

## 9. Evaluate Results

In this section, we'll demonstrate how to evaluate extraction results by comparing them with expected (ground truth) values. The evaluation process involves:

1. Creating a ground truth document with expected values
2. Comparing the actual extraction results against expected values
3. Calculating metrics (precision, recall, F1 score)
4. Generating an evaluation report

#### Evaluation helper function

In [ ]:
# Helper function to create a ground truth document from an existing document and expected results
def create_ground_truth_document(source_document, expected_results_dict):
    """Creates a ground truth document for evaluation from an existing document and expected results.
    
    Args:
        source_document: The original document to copy structure from
        expected_results_dict: Dictionary mapping section IDs to expected attribute values
        
    Returns:
        Document: A document with the same structure but with expected results
    """
    # Create a new document with the same core attributes
    ground_truth = Document(
        id=source_document.id,
        input_bucket=source_document.input_bucket,
        input_key=source_document.input_key,
        output_bucket=source_document.output_bucket,
        status=Status.COMPLETED
    )
    
    # Copy sections and add expected result URIs
    for section in source_document.sections:
        # Create section with same structure
        expected_section = Section(
            section_id=section.section_id,
            classification=section.classification,
            confidence=1.0,
            page_ids=section.page_ids.copy(),
            extraction_result_uri=section.extraction_result_uri  # Copy the URI from actual document
        )
        ground_truth.sections.append(expected_section)
    
    # Copy pages
    for page_id, page in source_document.pages.items():
        ground_truth.pages[page_id] = page
    
    # Store expected results to S3 for sections that have extraction results
    for section_id, expected_data in expected_results_dict.items():
        # Find the section in the document
        for section in ground_truth.sections:
            if section.section_id == section_id and section.extraction_result_uri:
                # Load the original extraction result as template
                uri = section.extraction_result_uri
                bucket, key = parse_s3_uri(uri)
                
                try:
                    # Get the original result structure
                    response = s3_client.get_object(Bucket=bucket, Key=key)
                    result_data = json.loads(response['Body'].read().decode('utf-8'))
                    
                    # Replace the inference_result with our expected data
                    if "inference_result" in result_data:
                        result_data["inference_result"] = expected_data
                    else:
                        # Or just replace the entire content if no inference_result key
                        result_data = expected_data
                    
                    # Write back to S3 with a different key for expected values
                    expected_key = key.replace("/result.json", "/expected.json")
                    s3_client.put_object(
                        Bucket=bucket,
                        Key=expected_key,
                        Body=json.dumps(result_data).encode('utf-8')
                    )
                    
                    # Update the section's extraction URI to point to our expected data
                    section.extraction_result_uri = f"s3://{bucket}/{expected_key}"
                    print(f"Stored expected results for section {section_id} at {section.extraction_result_uri}")
                except Exception as e:
                    print(f"Error storing expected results for section {section_id}: {e}")
    
    return ground_truth

#### Set up ground truth

In [ ]:
# Define expected results for extraction (ground truth)
# Customize values to showcase different evaluation methods from CONFIG
expected_results = {
    "1": {  # Section 1 (Letter)
        # For sender_name with LLM matching - intentionally create a variant that should match semantically
        "sender_name": "William E. Clarke",  
        # For sender_address with LLM matching - formatting differences should still match
        "sender_address": "206 maple Street\nP.O. Box 1056\nMurray Kentucky 42071-1056"  
    },
    "2": {  # Section 2 (Form)
        # For form_type with FUZZY matching (threshold 0.7) - added qualifier but should still match
        "form_type": "LAB SERVICES CONSISTENCY REPORT - Annual Edition",  
        # For form_id with NUMERIC_EXACT - should match
        "form_id": 2030053328  
    },
    "3": {  # Section 3 (Email)
        # For from_address with default matching (LLM) - match
        "from_address": "Kelahan, Benjamin",  
        # For to_address field with LLM matching
        "to_address": "TI Minnesota, TI New York"  
    }
}

# Create ground truth document using the helper function
expected_document = create_ground_truth_document(document, expected_results)


Stored expected results for section 1 at s3://idp-notebook-output-912625584728-us-west-2/sample-2025-05-27_14-24-41.pdf/sections/1/expected.json
Stored expected results for section 2 at s3://idp-notebook-output-912625584728-us-west-2/sample-2025-05-27_14-24-41.pdf/sections/2/expected.json
Stored expected results for section 3 at s3://idp-notebook-output-912625584728-us-west-2/sample-2025-05-27_14-24-41.pdf/sections/3/expected.json


#### Run evaluation

In [ ]:
# Create the evaluation service
evaluation_service = evaluation.EvaluationService(config=CONFIG)

# Run evaluation
print("Running document evaluation...")
start_time = time.time()
document = evaluation_service.evaluate_document(
    actual_document=document,
    expected_document=expected_document
)
evaluation_time = time.time() - start_time

print(f"Evaluation completed in {evaluation_time:.2f} seconds")
print(f"Evaluation report URI: {document.evaluation_report_uri}")

INFO:idp_common.evaluation.service:Initialized evaluation service with LLM configuration and max_workers=10
DEBUG:idp_common.evaluation.service:Evaluating Section 1 - class: letter, content: Section(section_id='1', classification='letter', confidence=1.0, page_ids=['1'], extraction_result_uri='s3://idp-notebook-output-912625584728-us-west-2/sample-2025-05-27_14-24-41.pdf/sections/1/result.json', attributes=None)
INFO:idp_common.evaluation.service:Comparing: sender_name using EvaluationMethod.LLM - from class letter
INFO:idp_common.evaluation.service:Comparing: sender_address using EvaluationMethod.LLM - from class letter
INFO:idp_common.evaluation.service:Comparing: recipient_name using EvaluationMethod.LLM - from class letter
INFO:idp_common.evaluation.service:Comparing: recipient_address using EvaluationMethod.LLM - from class letter


Running document evaluation...


INFO:idp_common.evaluation.service:Comparing: date using EvaluationMethod.LLM - from class letter
INFO:idp_common.evaluation.service:Comparing: subject using EvaluationMethod.LLM - from class letter
INFO:idp_common.evaluation.service:Comparing: reference_number using EvaluationMethod.LLM - from class letter
INFO:idp_common.evaluation.service:Comparing: signature using EvaluationMethod.LLM - from class letter
INFO:idp_common.evaluation.service:Comparing: cc using EvaluationMethod.LLM - from class letter
INFO:idp_common.evaluation.service:Comparing: letter_type using EvaluationMethod.LLM - from class letter
INFO:idp_common.bedrock.client:Bedrock request attempt 1/8:
DEBUG:idp_common.bedrock.client:  - model: us.anthropic.claude-3-5-sonnet-20241022-v2:0
DEBUG:idp_common.bedrock.client:  - inferenceConfig: {'temperature': 0.0}
DEBUG:idp_common.bedrock.client:  - system: [{'text': 'You are an evaluator that helps determine if the predicted and expected values match for document attribute ex

Evaluation completed in 5.84 seconds
Evaluation report URI: s3://idp-notebook-output-912625584728-us-west-2/sample-2025-05-27_14-24-41.pdf/evaluation/report.md


#### Display evaluation results

In [ ]:
# Show structured evaluation result
print("Evaluation result object")
if document.evaluation_result:
    print(f"{document.evaluation_result}")
else:
    print("ERROR.. No evaluation_result found")

# Read the evaluation report from S3
print("Reading markdown report from S3...")
if document.evaluation_report_uri:
    bucket, key = parse_s3_uri(document.evaluation_report_uri)
    response = s3_client.get_object(Bucket=bucket, Key=key)
    s3_markdown = response['Body'].read().decode('utf-8')
    print(f"Successfully read report from {document.evaluation_report_uri}")
else:
    print("No evaluation report URI found")

# Display the report in the notebook with proper formatting
from IPython.display import Markdown, display

# Display the markdown directly from S3 content
display(Markdown(s3_markdown))

Evaluation result object
DocumentEvaluationResult(document_id='rvl-cdip-package', section_results=[SectionEvaluationResult(section_id='1', document_class='letter', attributes=[AttributeEvaluationResult(name='cc', expected=None, actual=None, matched=True, score=1.0, reason='Both actual and expected values are missing, so they are matched.', error_details=None, evaluation_method='LLM', evaluation_threshold=None, comparator_type=None), AttributeEvaluationResult(name='date', expected=None, actual='10/31/1995', matched=False, score=0.0, reason=None, error_details=None, evaluation_method='LLM', evaluation_threshold=None, comparator_type=None), AttributeEvaluationResult(name='letter_type', expected=None, actual='opposition letter', matched=False, score=0.0, reason=None, error_details=None, evaluation_method='LLM', evaluation_threshold=None, comparator_type=None), AttributeEvaluationResult(name='recipient_address', expected=None, actual='United States Senate Washington, D. C. 20510', matched=F

# Document Evaluation: rvl-cdip-package

## Summary
- **Match Rate**: 🟠 20/32 attributes matched [████████████░░░░░░░░] 62%
- **Precision**: 0.29 | **Recall**: 0.67 | **F1 Score**: 🔴 0.40

## Overall Metrics
| Metric | Value | Rating |
| ------ | :----: | :----: |
| precision | 0.2857 | 🔴 Poor |
| recall | 0.6667 | 🟠 Fair |
| f1_score | 0.4000 | 🔴 Poor |
| accuracy | 0.6250 | 🟠 Fair |
| false_alarm_rate | 0.3846 | 🟠 Fair |
| false_discovery_rate | 0.0000 | 🟢 Excellent |


## Section: 1 (letter)
### Metrics
| Metric | Value | Rating |
| ------ | :----: | :----: |
| precision | 0.2500 | 🔴 Poor |
| recall | 1.0000 | 🟢 Excellent |
| f1_score | 0.4000 | 🔴 Poor |
| accuracy | 0.4000 | 🔴 Poor |
| false_alarm_rate | 0.7500 | 🔴 Poor |
| false_discovery_rate | 0.0000 | 🟢 Excellent |


### Attributes
| Status | Attribute | Expected | Actual | Score | Method | Reason |
| :----: | --------- | -------- | ------ | ----- | ------ | ------ |
| ✅ | cc | None | None | 1.00 | LLM | Both actual and expected values are missing, so they are matched. |
| ❌ | date | None | 10/31/1995 | 0.00 | LLM |  |
| ❌ | letter_type | None | opposition letter | 0.00 | LLM |  |
| ❌ | recipient_address | None | United States Senate Washington, D. C. 20510 | 0.00 | LLM |  |
| ❌ | recipient_name | None | The Honorable Wendell H. Ford | 0.00 | LLM |  |
| ❌ | reference_number | None | TNJB 0008497 | 0.00 | LLM |  |
| ✅ | sender_address | 206 maple Street P.O. Box 1056 Murray Kentucky 42071-1056 | 206 Maple Street P.O. Box 1056 Murray Kentucky 42071-1056 | 1.00 | LLM | The addresses are identical in content, only differing in formatting (line breaks vs. spaces). All components (street address, PO box, city, state, ZIP) match exactly. |
| ✅ | sender_name | William E. Clarke | Will E. Clark | 0.90 | LLM | The names are semantically equivalent - 'Will' is a common nickname/shortening of 'William', and 'Clark/Clarke' are common spelling variations of the same surname. The middle initial 'E.' matches exactly. |
| ❌ | signature | None | Will E. Clark | 0.00 | LLM |  |
| ✅ | subject | None | None | 1.00 | LLM | Both actual and expected values are missing, so they are matched. |


## Section: 2 (specification)
### Metrics
| Metric | Value | Rating |
| ------ | :----: | :----: |
| precision | 0.0000 | 🔴 Poor |
| recall | 0.0000 | 🔴 Poor |
| f1_score | 0.0000 | 🔴 Poor |
| accuracy | 0.7500 | 🟡 Good |
| false_alarm_rate | 0.1000 | 🟢 Excellent |
| false_discovery_rate | 0.0000 | 🟢 Excellent |


### Attributes
| Status | Attribute | Expected | Actual | Score | Method | Reason |
| :----: | --------- | -------- | ------ | ----- | ------ | ------ |
| ✅ | approval_info | None | None | 1.00 | LLM | Both actual and expected values are missing, so they are matched. |
| ✅ | compatibility | None | None | 1.00 | LLM | Both actual and expected values are missing, so they are matched. |
| ✅ | dimensions | None | None | 1.00 | LLM | Both actual and expected values are missing, so they are matched. |
| ❌ | form_id | 2030053328 | None | 0.00 | LLM | [Default method - attribute not specified in the configuration] |
| ❌ | form_type | LAB SERVICES CONSISTENCY REPORT - Annual Edition | None | 0.00 | LLM | [Default method - attribute not specified in the configuration] |
| ✅ | materials | None | None | 1.00 | LLM | Both actual and expected values are missing, so they are matched. |
| ✅ | product_name | None | None | 1.00 | LLM | Both actual and expected values are missing, so they are matched. |
| ✅ | requirements | None | None | 1.00 | LLM | Both actual and expected values are missing, so they are matched. |
| ✅ | revision_history | None | None | 1.00 | LLM | Both actual and expected values are missing, so they are matched. |
| ✅ | standards | None | None | 1.00 | LLM | Both actual and expected values are missing, so they are matched. |
| ❌ | technical_details | None | {'date': '07/28/93', 'technician': 'CC', 'shift': 'A', 'line': '2', 'area': '52', 'product_unit_code': '0728', 'sample_id': 'stuff box 2', 'reason_for_request': 'test', 'requested_delivery_time': None, 'time_sample_received': None, 'time_analysis_completed': None, 'data_communicated_to': 'Gone AT 105', 'drying_time': {'in': None, 'out': None}, 'consistency': {'average': '3.68', 'sample_10': '2.853', 'sample_11': '2.645', 'sample_12': '2.847', 'range': '.15'}, 'meter_reading': None, 'comments': None} | 0.00 | LLM |  |
| ✅ | version | None | None | 1.00 | LLM | Both actual and expected values are missing, so they are matched. |


## Section: 3 (email)
### Metrics
| Metric | Value | Rating |
| ------ | :----: | :----: |
| precision | 0.4000 | 🔴 Poor |
| recall | 1.0000 | 🟢 Excellent |
| f1_score | 0.5714 | 🟠 Fair |
| accuracy | 0.7000 | 🟡 Good |
| false_alarm_rate | 0.3750 | 🟠 Fair |
| false_discovery_rate | 0.0000 | 🟢 Excellent |


### Attributes
| Status | Attribute | Expected | Actual | Score | Method | Reason |
| :----: | --------- | -------- | ------ | ----- | ------ | ------ |
| ✅ | attachments | None | None | 1.00 | LLM | Both actual and expected values are missing, so they are matched. |
| ✅ | bcc_address | None | None | 1.00 | LLM | Both actual and expected values are missing, so they are matched. |
| ❌ | cc_address | None | Ashley Bratich (MSMAIL) | 0.00 | LLM |  |
| ❌ | date_sent | None | 04/18/1998 | 0.00 | LLM |  |
| ✅ | from_address | Kelahan, Benjamin | Kelahan, Ben | 0.95 | LLM | Benjamin and Ben are common variations of the same name, with Ben being a standard nickname/shortened form of Benjamin. The last name Kelahan is identical in both cases. |
| ✅ | message_id | None | None | 1.00 | LLM | Both actual and expected values are missing, so they are matched. |
| ✅ | priority | None | None | 1.00 | LLM | Both actual and expected values are missing, so they are matched. |
| ❌ | subject | None | FW: Morning Team Notes 4/20 | 0.00 | LLM |  |
| ✅ | thread_id | None | None | 1.00 | LLM | Both actual and expected values are missing, so they are matched. |
| ✅ | to_address | TI Minnesota, TI New York | TI New York: 'TI Minnesota | 0.95 | LLM | Both values contain 'TI Minnesota' and 'TI New York' as recipients, just in slightly different order and formatting. The semantic meaning and core information is the same despite minor formatting differences. |


Execution time: 5.54 seconds

## Evaluation Methods Used

This evaluation used the following methods to compare expected and actual values:

1. **EXACT** - Exact string match after stripping punctuation and whitespace
2. **NUMERIC_EXACT** - Exact numeric match after normalizing
3. **FUZZY** - Fuzzy string matching using string similarity metrics (with evaluation_threshold)
4. **SEMANTIC** - Semantic similarity comparison using Bedrock Titan embeddings (with evaluation_threshold)
5. **HUNGARIAN** - Bipartite matching algorithm for lists of values
   - **EXACT** - Hungarian matching with exact string comparison
   - **FUZZY** - Hungarian matching with fuzzy string comparison (with evaluation_threshold)
   - **NUMERIC** - Hungarian matching with numeric comparison
6. **LLM** - Advanced semantic evaluation using Bedrock large language models

Each attribute is configured with a specific evaluation method based on the data type and comparison needs.

# 10. Clean Up (Optional)

In [ ]:
# Function to delete objects in a bucket
def delete_bucket_objects(bucket_name):
    try:
        # List all objects in the bucket
        response = s3_client.list_objects_v2(Bucket=bucket_name)
        if 'Contents' in response:
            delete_keys = {'Objects': [{'Key': obj['Key']} for obj in response['Contents']]}
            s3_client.delete_objects(Bucket=bucket_name, Delete=delete_keys)
            print(f"Deleted all objects in bucket {bucket_name}")
        else:
            print(f"Bucket {bucket_name} is already empty")
            
        # Delete bucket
        s3_client.delete_bucket(Bucket=bucket_name)
        print(f"Deleted bucket {bucket_name}")
    except Exception as e:
        print(f"Error cleaning up bucket {bucket_name}: {str(e)}")

# Uncomment the following lines to delete the buckets
# print("Cleaning up resources...")
# delete_bucket_objects(input_bucket_name)
# delete_bucket_objects(output_bucket_name)
# print("Cleanup complete")

## Conclusion

This notebook demonstrates the end-to-end processing flow using AWS services and the unified Document model:

1. **Document Creation** - Initialize a Document object with input/output locations
2. **OCR Processing** - Convert PDF to text using AWS Textract via OcrService
3. **Classification** - Identify document types and sections with Claude via ClassificationService
4. **Extraction** - Extract structured information with Claude via ExtractionService
5. **Evaluation** - Compare extraction results against expected values and generate metrics
6. **Document Model** - Document object is consistently used between all services
7. **Result Storage** - Extraction results are stored in S3 with URIs tracked in the Document

Key benefits of this approach:

1. **Modularity** - Each service has a clear responsibility
2. **Consistency** - Same data model flows through the entire pipeline
3. **Performance** - Focused document pattern reduces resource usage
4. **Flexibility** - Support for multiple backends (Bedrock, SageMaker)
5. **Maintainability** - Standardized patterns across services
6. **Measurement** - Built-in evaluation capabilities to measure accuracy

This example uses a  workflow with:
1. S3 buckets (created specifically for this demo)
2. AWS Textract OCR processing
3. LLM inferencing via Bedrock
4. A document sample (rvl_cdip_package.pdf)

The Evaluation Service specifically provides:

1. Multiple evaluation methods (EXACT, NUMERIC_EXACT, FUZZY)
2. Per-attribute and document-level metrics
3. Markdown and JSON format reporting
4. Integration with the Document model
5. Configuration-driven evaluation methods